Baseline одного из решений

In [3]:
# импортируем библиотеки
import pandas as pd
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing

In [6]:
df = pd.read_csv('data/train_geo.csv')
df.head(10)

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,OFFSHORE,OIL,DEVELOPING,INVERSION/WRENCH,3520,NEOGENE,SANDSTONE,2460.0,220.0,20.0,45.0
1,GRAVITY/EXTENSION/EVAPORITE,OFFSHORE,OIL,MATURE PRODUCTION,SALT/PASSIVE MARGIN,9967,CRETACEOUS,LIMESTONE,427.0,160.0,19.0,175.0
2,GRAVITY/EXTENSION/EVAPORITE,ONSHORE,OIL,MATURE PRODUCTION,PASSIVE MARGIN,8700,CRETACEOUS,LIMESTONE,95.0,15.0,12.0,20.0
3,COMPRESSION,ONSHORE,OIL,DECLINING PRODUCTION,THRUST,5084,CRETACEOUS,SANDSTONE,328.0,300.0,13.0,600.0
4,INVERSION/COMPRESSION/EXTENSION,ONSHORE,OIL,DECLINING PRODUCTION,INVERSION/RIFT,1030,CRETACEOUS,SANDSTONE,260.0,33.0,24.0,182.0
5,COMPRESSION/EXTENSION/EVAPORITE,ONSHORE,OIL,DECLINING PRODUCTION,INTRACRATONIC,5575,CARBONIFEROUS,DOLOMITE,80.0,46.0,14.0,15.0
6,INVERSION/COMPRESSION/EXTENSION,ONSHORE,OIL,DEVELOPING,INVERSION/RIFT,5216,PROTEROZOIC,SANDSTONE,200.0,25.0,10.0,209.0
7,INVERSION/COMPRESSION/EXTENSION,OFFSHORE,OIL,DEVELOPING,RIFT,8100,CRETACEOUS,SANDSTONE,115.0,40.0,15.0,30.0
8,COMPRESSION,ONSHORE,OIL,DECLINING PRODUCTION,INTRACRATONIC,1915,CARBONIFEROUS,LIMESTONE,330.0,20.0,10.0,35.0
9,COMPRESSION,ONSHORE,OIL,DECLINING PRODUCTION,FORELAND,4150,PERMIAN,DOLOMITE,225.0,200.0,17.5,62.0


In [7]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [9]:
X_test = pd.read_csv('data/test_geo.csv')
X_test.head(10)

,Tectonic regime,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,COMPRESSION,GAS,MATURE PRODUCTION,FORELAND,2275,PERMIAN,SANDSTONE,325.0,30.0,13.0,0.04
1,INVERSION/COMPRESSION/EXTENSION/EROSION,OIL,ABANDONED,FORELAND,500,PERMIAN,SANDSTONE,40.0,18.0,28.0,240.00
2,INVERSION/COMPRESSION/EXTENSION/EVAPORITE,GAS,DECLINING PRODUCTION,INVERSION/RIFT,5548,JURASSIC,DOLOMITE,200.0,20.0,13.0,7.30
3,COMPRESSION/EVAPORITE/EXTENSION/LINKED,OIL,DEVELOPING,THRUST,10100,NEOGENE,SANDSTONE,8200.0,260.0,18.0,100.00
4,COMPRESSION/EROSION,GAS-CONDENSATE,MATURE PRODUCTION,FORELAND,8750,CARBONIFEROUS,DOLOMITE,140.0,70.0,12.0,125.00
5,EXTENSION,GAS-CONDENSATE,DECLINING PRODUCTION,RIFT,7546,CRETACEOUS,SANDSTONE,361.0,148.0,20.0,225.00
6,COMPRESSION,OIL,DECLINING PRODUCTION,INTRACRATONIC,9125,CARBONIFEROUS,DOLOMITE,105.0,40.0,16.0,25.00
7,COMPRESSION,OIL,SECOND PLATEAU PRODUTION,INTRACRATONIC,4300,CARBONIFEROUS,DOLOMITE,82.0,33.0,15.0,10.00
8,COMPRESSION,GAS,DECLINING PRODUCTION,INTRACRATONIC,1305,CRETACEOUS,SILTSTONE,30.0,10.7,20.0,2.00
9,EXTENSION,OIL,MATURE PRODUCTION,RIFT,3898,NEOGENE,SANDSTONE,2615.0,66.0,31.0,1475.00


In [10]:
y_test = pd.read_csv('data/answer.csv', header=None)
y_test.columns = ['Onshore/Offshore']

In [11]:
label_list = []
iter_cols = ['Structural setting','Period', 'Lithology',
             'Tectonic regime', 'Hydrocarbon type','Reservoir status']
for column in iter_cols:
    le = preprocessing.LabelEncoder()
    df[column] = le.fit_transform(df[column].values)
    X_test[column] = le.fit_transform(X_test[column].values)

In [12]:
le = preprocessing.LabelEncoder()
df['Onshore/Offshore'] = le.fit_transform(df['Onshore/Offshore'].values)
y_test['Onshore/Offshore'] = le.fit_transform(y_test['Onshore/Offshore'].values)

In [13]:
df.head(10)

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,51,0,4,4,19,3520,12,10,2460.0,220.0,20.0,45.0
1,33,0,4,5,28,9967,6,8,427.0,160.0,19.0,175.0
2,33,1,4,5,20,8700,6,8,95.0,15.0,12.0,20.0
3,0,1,4,2,37,5084,6,10,328.0,300.0,13.0,600.0
4,41,1,4,2,17,1030,6,10,260.0,33.0,24.0,182.0
5,10,1,4,2,12,5575,3,6,80.0,46.0,14.0,15.0
6,41,1,4,4,17,5216,18,10,200.0,25.0,10.0,209.0
7,41,0,4,4,21,8100,6,10,115.0,40.0,15.0,30.0
8,0,1,4,2,12,1915,3,8,330.0,20.0,10.0,35.0
9,0,1,4,2,7,4150,17,6,225.0,200.0,17.5,62.0


In [14]:
X_train = df[['Structural setting','Period', 'Lithology',
              'Tectonic regime', 'Hydrocarbon type','Reservoir status',
              'Depth','Gross',	'Netpay',	'Porosity',	'Permeability']]
y_train = df['Onshore/Offshore']

In [15]:
model = GradientBoostingClassifier(random_state=0)
model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [16]:
accuracy_score(y_test, model.predict(X_test))

/opt/anaconda3/envs/yandex_я_проф/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


0.6992481203007519

In [17]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')

In [18]:
model.fit(X_train,y_train)

/opt/anaconda3/envs/yandex_я_проф/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 0.5673254	total: 72.7ms	remaining: 1m 12s
1:	learn: 0.4725040	total: 74.5ms	remaining: 37.2s
2:	learn: 0.4546794	total: 75.1ms	remaining: 25s
3:	learn: 0.4236528	total: 75.6ms	remaining: 18.8s
4:	learn: 0.3924106	total: 76.1ms	remaining: 15.1s
5:	learn: 0.3840466	total: 76.5ms	remaining: 12.7s
6:	learn: 0.3815121	total: 77ms	remaining: 10.9s
7:	learn: 0.3762118	total: 77.6ms	remaining: 9.62s
8:	learn: 0.3706895	total: 78.2ms	remaining: 8.61s
9:	learn: 0.3526562	total: 79.7ms	remaining: 7.89s
10:	learn: 0.3478919	total: 80.5ms	remaining: 7.24s
11:	learn: 0.3226570	total: 81ms	remaining: 6.67s
12:	learn: 0.3139459	total: 81.5ms	remaining: 6.19s
13:	learn: 0.3082388	total: 82.1ms	remaining: 5.78s
14:	learn: 0.2989984	total: 82.6ms	remaining: 5.42s
15:	learn: 0.2892681	total: 83.1ms	remaining: 5.11s
16:	learn: 0.2833878	total: 83.6ms	remaining: 4.83s
17:	learn: 0.2812327	total: 84.2ms	remaining: 4.59s
18:	learn: 0.2674314	total: 84.9ms	remaining: 4.38s
19:	learn: 0.2667010	total:

In [19]:
accuracy_score(y_test, model.predict(X_test))

/opt/anaconda3/envs/yandex_я_проф/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0.7518796992481203